In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_wudao_zonghe=pd.DataFrame(columns=columns)
channel_wudao_zonghe

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[5]/div/a/span').click()  #选择“舞蹈”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[5]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'舞蹈'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[6]/a').click()  #选择“宅舞”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[6]/a').text
sub_channel

'舞蹈综合'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [9]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_wudao_zonghe.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-20 19:34:14.600259


<ipython-input-9-201e32c10b0d>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-201e32c10b0d>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-201e32c10b0d>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1oJ411J7xx视频属性的抓取
BV号为1oJ411J7xx的视频属性抓取结束


<ipython-input-9-201e32c10b0d>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1QJ411p74k视频属性的抓取
BV号为1QJ411p74k的视频属性抓取结束
开始BV号为1xJ411V7u3视频属性的抓取
BV号为1xJ411V7u3的视频属性抓取结束
开始BV号为1a7411r7kb视频属性的抓取
BV号为1a7411r7kb的视频属性抓取结束
开始BV号为1rJ411b7w6视频属性的抓取
BV号为1rJ411b7w6的视频属性抓取结束
开始BV号为1W7411k7fd视频属性的抓取
BV号为1W7411k7fd的视频属性抓取结束
开始BV号为1ZJ41177VZ视频属性的抓取
BV号为1ZJ41177VZ的视频属性抓取结束
开始BV号为1oJ411j7C5视频属性的抓取
BV号为1oJ411j7C5的视频属性抓取结束
开始BV号为1CJ411E7ha视频属性的抓取
BV号为1CJ411E7ha的视频属性抓取结束
开始BV号为1PJ411J7uT视频属性的抓取
BV号为1PJ411J7uT的视频属性抓取结束
开始BV号为1T7411v7UK视频属性的抓取
BV号为1T7411v7UK的视频属性抓取结束
开始BV号为1c7411C7jw视频属性的抓取
BV号为1c7411C7jw的视频属性抓取结束
开始BV号为15J411L78Y视频属性的抓取
BV号为15J411L78Y的视频属性抓取结束
开始BV号为1jJ411L7uq视频属性的抓取
BV号为1jJ411L7uq的视频属性抓取结束
开始BV号为1sJ411V7gF视频属性的抓取
BV号为1sJ411V7gF的视频属性抓取结束
开始BV号为147411a7GF视频属性的抓取
BV号为147411a7GF的视频属性抓取结束
开始BV号为1S7411B73m视频属性的抓取
BV号为1S7411B73m的视频属性抓取结束
开始BV号为11J411L7qC视频属性的抓取
BV号为11J411L7qC的视频属性抓取结束
开始BV号为1f7411r7En视频属性的抓取
BV号为1f7411r7En的视频属性抓取结束
开始BV号为1b7411B7b8视频属性的抓取
BV号为1b7411B7b8的视频属性抓取结束
开始BV号为1KJ411C7tp视频属性的抓取
BV号为1KJ411C7tp的视频属性抓取结束
开始BV号为1X7411e7WH视频属性的抓取
BV号为1X7411e7WH的视

...

开始BV号为1H7411x7GH视频属性的抓取
BV号为1H7411x7GH的视频属性抓取结束
开始BV号为1G7411c7qy视频属性的抓取
BV号为1G7411c7qy的视频属性抓取结束
开始BV号为1U7411A7oc视频属性的抓取
BV号为1U7411A7oc的视频属性抓取结束
开始BV号为137411t75B视频属性的抓取
BV号为137411t75B的视频属性抓取结束
开始BV号为1X7411H7Kh视频属性的抓取
BV号为1X7411H7Kh的视频属性抓取结束
开始BV号为1Q7411F7qy视频属性的抓取
BV号为1Q7411F7qy的视频属性抓取结束
开始BV号为1j7411h7fv视频属性的抓取
BV号为1j7411h7fv的视频属性抓取结束
开始BV号为117411F7uN视频属性的抓取
BV号为117411F7uN的视频属性抓取结束
开始BV号为137411j7cU视频属性的抓取
BV号为137411j7cU的视频属性抓取结束
开始BV号为14E411J7Y9视频属性的抓取
BV号为14E411J7Y9的视频属性抓取结束
开始BV号为1r7411L7RU视频属性的抓取
BV号为1r7411L7RU的视频属性抓取结束
开始BV号为1q7411c7Ya视频属性的抓取
BV号为1q7411c7Ya的视频属性抓取结束
开始BV号为1L7411g78Y视频属性的抓取
BV号为1L7411g78Y的视频属性抓取结束
开始BV号为177411J7pC视频属性的抓取
BV号为177411J7pC的视频属性抓取结束
开始BV号为1n7411L7UG视频属性的抓取
BV号为1n7411L7UG的视频属性抓取结束
开始BV号为1u7411A7JS视频属性的抓取
BV号为1u7411A7JS的视频属性抓取结束
开始BV号为1x7411J7Rk视频属性的抓取
BV号为1x7411J7Rk的视频属性抓取结束
开始BV号为1f7411H7RR视频属性的抓取
BV号为1f7411H7RR的视频属性抓取结束
开始BV号为1D7411A7PU视频属性的抓取
BV号为1D7411A7PU的视频属性抓取结束
开始BV号为1G7411x7SQ视频属性的抓取
BV号为1G7411x7SQ的视频属性抓取结束
开始BV号为1w741117Ui视频属性的抓取
BV号为1w741117Ui的视

开始BV号为1A5411r79B视频属性的抓取
BV号为1A5411r79B的视频属性抓取结束
开始BV号为1Xf4y1S7xo视频属性的抓取
BV号为1Xf4y1S7xo的视频属性抓取结束
开始BV号为1KZ4y147Vr视频属性的抓取
BV号为1KZ4y147Vr的视频属性抓取结束
开始BV号为1Cg4y187vt视频属性的抓取
BV号为1Cg4y187vt的视频属性抓取结束
开始BV号为1Yf4y1U7iT视频属性的抓取
BV号为1Yf4y1U7iT的视频属性抓取结束
开始BV号为1qf4y1S7K8视频属性的抓取
BV号为1qf4y1S7K8的视频属性抓取结束
开始BV号为1Yp4y1X76a视频属性的抓取
BV号为1Yp4y1X76a的视频属性抓取结束
开始BV号为1Cp4y117rL视频属性的抓取
BV号为1Cp4y117rL的视频属性抓取结束
开始BV号为1ZZ4y1x7PM视频属性的抓取
BV号为1ZZ4y1x7PM的视频属性抓取结束
开始BV号为1Zt4y127Un视频属性的抓取
BV号为1Zt4y127Un的视频属性抓取结束
开始BV号为1AT4y137L4视频属性的抓取
BV号为1AT4y137L4的视频属性抓取结束
开始BV号为1Bk4y197sL视频属性的抓取
BV号为1Bk4y197sL的视频属性抓取结束
开始BV号为19C4y1s7SY视频属性的抓取
BV号为19C4y1s7SY的视频属性抓取结束
开始BV号为1A5411t73C视频属性的抓取
BV号为1A5411t73C的视频属性抓取结束
开始BV号为1zK41157gD视频属性的抓取
BV号为1zK41157gD的视频属性抓取结束
开始BV号为1iZ4y1x7iK视频属性的抓取
BV号为1iZ4y1x7iK的视频属性抓取结束
开始BV号为1o54y197Kq视频属性的抓取
BV号为1o54y197Kq的视频属性抓取结束
开始BV号为1Bg4y187xW视频属性的抓取
BV号为1Bg4y187xW的视频属性抓取结束
开始BV号为1XT4y137e6视频属性的抓取
BV号为1XT4y137e6的视频属性抓取结束
开始BV号为1VQ4y1K7uS视频属性的抓取
BV号为1VQ4y1K7uS的视频属性抓取结束
开始BV号为1454y1976e视频属性的抓取
BV号为1454y1976e的视

开始BV号为17Z4y1H72g视频属性的抓取
BV号为17Z4y1H72g的视频属性抓取结束
开始BV号为1Jg4y1v7Kw视频属性的抓取
BV号为1Jg4y1v7Kw的视频属性抓取结束
开始BV号为1RZ4y1H72k视频属性的抓取
BV号为1RZ4y1H72k的视频属性抓取结束
开始BV号为12z4y197Lv视频属性的抓取
BV号为12z4y197Lv的视频属性抓取结束
开始BV号为1Y54y1B7uw视频属性的抓取
BV号为1Y54y1B7uw的视频属性抓取结束
开始BV号为1Vv411B7aW视频属性的抓取
开始BV号为1Vv411B7aW视频属性的抓取
BV号为1Vv411B7aW的视频属性抓取结束
开始BV号为1qV411r7bU视频属性的抓取
BV号为1qV411r7bU的视频属性抓取结束
开始BV号为1Sk4y1B7mA视频属性的抓取
BV号为1Sk4y1B7mA的视频属性抓取结束
开始BV号为1Wz4y1971F视频属性的抓取
BV号为1Wz4y1971F的视频属性抓取结束
开始BV号为1wK411H7LA视频属性的抓取
BV号为1wK411H7LA的视频属性抓取结束
开始BV号为1Dv411B7aG视频属性的抓取
BV号为1Dv411B7aG的视频属性抓取结束
开始BV号为1Ag4y1q7FW视频属性的抓取
BV号为1Ag4y1q7FW的视频属性抓取结束
开始BV号为1Zv411q75K视频属性的抓取
BV号为1Zv411q75K的视频属性抓取结束
开始BV号为1xz4y1d7Mm视频属性的抓取
BV号为1xz4y1d7Mm的视频属性抓取结束
开始BV号为1EZ4y1H7Uk视频属性的抓取
BV号为1EZ4y1H7Uk的视频属性抓取结束
开始BV号为14K4y1t78N视频属性的抓取
BV号为14K4y1t78N的视频属性抓取结束
开始BV号为1eg4y1i7w9视频属性的抓取
BV号为1eg4y1i7w9的视频属性抓取结束
开始BV号为1Lz4y1X7t5视频属性的抓取
BV号为1Lz4y1X7t5的视频属性抓取结束
开始BV号为13a4y1v76b视频属性的抓取
BV号为13a4y1v76b的视频属性抓取结束
开始BV号为1sz4y1X7yj视频属性的抓取
BV号为1sz4y1X7yj的视频属性抓取结束
开始BV号为1J5411W7kU

开始BV号为11V411B7ts视频属性的抓取
BV号为11V411B7ts的视频属性抓取结束
开始BV号为1pA411Y7w9视频属性的抓取
BV号为1pA411Y7w9的视频属性抓取结束
开始BV号为1sV411z7Bv视频属性的抓取
BV号为1sV411z7Bv的视频属性抓取结束
开始BV号为1Vv411q7Rg视频属性的抓取
BV号为1Vv411q7Rg的视频属性抓取结束
开始BV号为1YA411e7BH视频属性的抓取
BV号为1YA411e7BH的视频属性抓取结束
开始BV号为1ET4y1E7yz视频属性的抓取
BV号为1ET4y1E7yz的视频属性抓取结束
开始BV号为1wv411q757视频属性的抓取
BV号为1wv411q757的视频属性抓取结束
开始BV号为16v411q7rQ视频属性的抓取
BV号为16v411q7rQ的视频属性抓取结束
开始BV号为1pz4y1f7TP视频属性的抓取
BV号为1pz4y1f7TP的视频属性抓取结束
开始BV号为1FA411Y7wh视频属性的抓取
BV号为1FA411Y7wh的视频属性抓取结束
开始BV号为16A411n7hU视频属性的抓取
BV号为16A411n7hU的视频属性抓取结束
开始BV号为1x54y1e791视频属性的抓取
BV号为1x54y1e791的视频属性抓取结束
开始BV号为1D64y1c7cG视频属性的抓取
BV号为1D64y1c7cG的视频属性抓取结束
开始BV号为1Sa4y1E75P视频属性的抓取
BV号为1Sa4y1E75P的视频属性抓取结束
开始BV号为1W5411h7fk视频属性的抓取
开始BV号为1W5411h7fk视频属性的抓取
BV号为1W5411h7fk的视频属性抓取结束
开始BV号为1Lp4y1q77q视频属性的抓取
BV号为1Lp4y1q77q的视频属性抓取结束
开始BV号为1Bf4y197EZ视频属性的抓取
BV号为1Bf4y197EZ的视频属性抓取结束
开始BV号为1Fv411i7Wb视频属性的抓取
开始BV号为1Fv411i7Wb视频属性的抓取
开始BV号为1Fv411i7Wb视频属性的抓取
BV号为1Fv411i7Wb的视频属性抓取结束
开始BV号为1YA411n7CX视频属性的抓取
BV号为1YA411n7CX的视频属性抓取结束
开始BV号为1t5411a7Bw

BV号为1F541187Bo的视频属性抓取结束
开始BV号为1kv41117bM视频属性的抓取
BV号为1kv41117bM的视频属性抓取结束
开始BV号为1SK411N7pU视频属性的抓取
BV号为1SK411N7pU的视频属性抓取结束
开始BV号为1cy4y1y7ap视频属性的抓取
BV号为1cy4y1y7ap的视频属性抓取结束
开始BV号为1tk4y1C7rL视频属性的抓取
开始BV号为1tk4y1C7rL视频属性的抓取
BV号为1tk4y1C7rL的视频属性抓取结束
开始BV号为1PA411J7p4视频属性的抓取
BV号为1PA411J7p4的视频属性抓取结束
开始BV号为16K411P7MY视频属性的抓取
BV号为16K411P7MY的视频属性抓取结束
开始BV号为1hi4y1u76V视频属性的抓取
BV号为1hi4y1u76V的视频属性抓取结束
开始BV号为1vt4y1S7TZ视频属性的抓取
BV号为1vt4y1S7TZ的视频属性抓取结束
开始BV号为1fD4y1o7kH视频属性的抓取
BV号为1fD4y1o7kH的视频属性抓取结束
开始BV号为14a4y1j73q视频属性的抓取
BV号为14a4y1j73q的视频属性抓取结束
开始BV号为1bi4y1u7V3视频属性的抓取
BV号为1bi4y1u7V3的视频属性抓取结束
开始BV号为17Z4y157wn视频属性的抓取
BV号为17Z4y157wn的视频属性抓取结束
开始BV号为1Vf4y1X7qJ视频属性的抓取
BV号为1Vf4y1X7qJ的视频属性抓取结束
开始BV号为1Di4y1M7vv视频属性的抓取
BV号为1Di4y1M7vv的视频属性抓取结束
开始BV号为1df4y1S7uq视频属性的抓取
BV号为1df4y1S7uq的视频属性抓取结束
开始BV号为17p4y1i79i视频属性的抓取
BV号为17p4y1i79i的视频属性抓取结束
开始BV号为1Ci4y1M7s9视频属性的抓取
BV号为1Ci4y1M7s9的视频属性抓取结束
开始BV号为1kK4y1a7uH视频属性的抓取
BV号为1kK4y1a7uH的视频属性抓取结束
开始BV号为1Jf4y1D76c视频属性的抓取
BV号为1Jf4y1D76c的视频属性抓取结束
开始BV号为1bp4y1a7MU视频属性的抓取
BV号为1bp4y1a7MU的视

In [27]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-27-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [10]:
channel_wudao_zonghe.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
2月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
3月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
4月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [11]:
channel_wudao_zonghe.to_json('channel_wudao_zonghe.json')